# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

t = Time("2024-12-01 00:00")

In [14]:
from jorbit.particle import Particle
from jorbit.system import System

obj = Horizons(id="274301", location="@0", epochs=t.tdb.jd)
vecs = obj.vectors(refplane="earth")

x = jnp.array([vecs["x"][0], vecs["y"][0], vecs["z"][0]])
v = jnp.array([vecs["vx"][0], vecs["vy"][0], vecs["vz"][0]])

p = Particle(x=x, v=v, time=t, name="wiki")
s = System(particles=[p], acceleration_func="newtonian planets")

print(s.acceleration_func(s.state))

s

[[ 2.61030935e-05 -2.90119924e-05 -8.52860259e-06]]


*************
jorbit System
 time: 2460645.50080073
 particles: [Particle: wiki]
*************

In [6]:
from jorbit.data.constants import (
    DEFAULT_PLANET_EPHEMERIS_URL,
    DEFAULT_ASTEROID_EPHEMERIS_URL,
    ALL_PLANET_IDS,
    ALL_PLANET_LOG_GMS,
    LARGE_ASTEROID_IDS,
    LARGE_ASTEROID_LOG_GMS,
)
from astropy.utils.data import download_file
from jplephem.spk import SPK

kernel = SPK.open(download_file(DEFAULT_ASTEROID_EPHEMERIS_URL, cache=True))

In [10]:
kernel = SPK.open(download_file(DEFAULT_PLANET_EPHEMERIS_URL, cache=True))
kernel = SPK.open(download_file(DEFAULT_ASTEROID_EPHEMERIS_URL, cache=True))
for s in kernel.segments:
    print(s.describe())

-8000-12-21..-3619-08-16  Type 2  Sun (10) -> Unknown Target (2000107)
  frame=1 source=MONTE Chebyshev Polynomial Table
-3619-08-16..762-04-11  Type 2  Sun (10) -> Unknown Target (2000107)
  frame=1 source=MONTE Chebyshev Polynomial Table
762-04-11..5142-12-05  Type 2  Sun (10) -> Unknown Target (2000107)
  frame=1 source=MONTE Chebyshev Polynomial Table
5142-12-05..9000-01-01  Type 2  Sun (10) -> Unknown Target (2000107)
  frame=1 source=MONTE Chebyshev Polynomial Table
-8000-12-21..-3619-08-16  Type 2  Sun (10) -> Ceres (2000001)
  frame=1 source=MONTE Chebyshev Polynomial Table
-3619-08-16..762-04-11  Type 2  Sun (10) -> Ceres (2000001)
  frame=1 source=MONTE Chebyshev Polynomial Table
762-04-11..5142-12-05  Type 2  Sun (10) -> Ceres (2000001)
  frame=1 source=MONTE Chebyshev Polynomial Table
5142-12-05..9000-01-01  Type 2  Sun (10) -> Ceres (2000001)
  frame=1 source=MONTE Chebyshev Polynomial Table
-8000-12-21..-3619-08-16  Type 2  Sun (10) -> Unknown Target (2000065)
  frame=1 s

In [ ]:
from jorbit.accelerations import create_newtonian_ephemeris_acceleration_func

eph.FunctionalEphemeris

q = create_newtonian_ephemeris_acceleration_func(eph.FunctionalEphemeris)

q(s.state)

Array([[ 2.61030935e-05, -2.90119924e-05, -8.52860259e-06]], dtype=float64)

In [2]:
from jorbit.observation import Observations

obj = Horizons(id="274301", location="695", epochs=t.tdb.jd)
eph = obj.ephemerides(extra_precision=True, quantities="1")
coord = SkyCoord(eph["RA"][0], eph["DEC"][0], unit=(u.deg, u.deg), frame="icrs")

obs = Observations(
    observed_coordinates=coord,
    times=t,
    observatories="695@399",
    astrometric_uncertainties=1 * u.arcsec,
    verbose=True,
)
obs

Observations with 1 set(s) of observations

In [7]:
eph.FunctionalEphemeris.log_gms

Array([-23.73665301, -21.045753  , -20.82895848, -23.07194211,
       -15.07946488, -16.28536632, -18.16446878, -17.99910783,
       -26.8539481 ,  -8.12544774], dtype=float64)

In [8]:
Time("J2000", format="jyear_str").iso

'2000-01-01 12:00:00.000'

In [9]:
Time(0.0, format="stardate").iso

'2318-07-05 11:00:00.000'